#Last second wala final kia h fir apn ne


In [7]:
from google.colab import userdata
import pandas as pd
import numpy as np
import logging
import random
import threading
import sys
import time
from typing import TypedDict, List

# Import LangGraph and related components
from langgraph.graph import END, StateGraph
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# Setup logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --------------------------------------------------------------------
# Data Loading and Document Preparation
# --------------------------------------------------------------------
groq_api = userdata.get("groq_api_key")
df = pd.read_csv("/content/defects.csv")             # Contains defects and solutions
test_cases_df = pd.read_csv("/content/test_cases.csv")  # Contains test cases

docs = []
for _, row in df.iterrows():
    if pd.notna(row["Description"]) and pd.notna(row["Solution"]):
        docs.append(Document(
            page_content=row["Description"],
            metadata={
                "solution": row["Solution"],
                "module": row["Module"]
            }
        ))

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# --------------------------------------------------------------------
# Define Agent State and LLM Initialization
# --------------------------------------------------------------------
class AgentState(TypedDict):
    input: str
    context: List[Document]
    response: str

llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.3,
    model_name="gemma2-9b-it",
)

# --------------------------------------------------------------------
# Helper: Classify Test Case (Positive/Negative) with Enhanced Keywords
# --------------------------------------------------------------------
def classify_test_case(tc_text: str) -> str:
    negative_keywords = [
        "fail", "error", "misconfigured", "incorrect", "doesn't work",
        "not work", "negative", "invalid", "wrong", "missing", "unexpected",
        "should not", "incorrectly", "failure", "reject", "malformed",
        "timeout", "invalid input", "edge case", "out of bounds"
    ]
    text_lower = tc_text.lower()
    return "negative" if any(kw in text_lower for kw in negative_keywords) else "positive"

# --------------------------------------------------------------------
# Workflow Node: Validate or Generate Test Cases (Enhanced to force generation)
# --------------------------------------------------------------------
def validate_or_generate_test_cases(state: AgentState):
    try:
        # Even if CSV test cases exist, force generation of 2 positive and 2 negative test cases.
        if not state["context"]:
            return {"response": "**Error**: The defect could not be found in the database."}
        context = state["context"][0]
        error_message = state["input"]
        solution = context.metadata["solution"]
        module = context.metadata["module"]

        # Generate explanation for the solution
        explanation_prompt = """
        [INST] Explain why this solution fixes the following error:
        Error: {error}
        Solution: {solution}
        [/INST]
        """
        explanation_template = ChatPromptTemplate.from_template(explanation_prompt)
        formatted_explanation = explanation_template.format_prompt(error=error_message, solution=solution)
        explanation = llm.invoke(formatted_explanation.to_messages()).content.strip()

        # --- Force Generation of Test Cases ---
        required_pos = 2
        required_neg = 2

        # Generate positive test cases
        pos_prompt = """
        [INST] Generate EXACTLY {count} POSITIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST:
        - Validate correct functionality under NORMAL conditions.
        - Include **Test Scenario**, **Test Steps**, **Pre Requisites**, **Expected Results**, **Pass/Fail Criteria**.
        - Be numbered (e.g., 1., 2.) and separated by two newlines.

        Example:
        1. **Test Scenario**: Valid configuration test
        **Test Steps**: 1. Configure system properly 2. Execute feature
        **Pre Requisites**: System updated
        **Expected Results**: Feature works without errors
        **Pass/Fail Criteria**: Pass if no errors observed

        [/INST]
        """.format(error=error_message, solution=solution, count=required_pos)
        pos_response = llm.invoke(ChatPromptTemplate.from_template(pos_prompt).format_prompt().to_messages()).content.strip()
        generated_pos = [tc.strip() for tc in pos_response.split("\n\n") if tc.strip()]
        generated_pos = generated_pos[:required_pos]  # Ensure exactly required count

        # Generate negative test cases
        neg_prompt = """
        [INST] Generate EXACTLY {count} NEGATIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST:
        - Validate error handling or invalid configurations.
        - Include **Test Scenario**, **Test Steps**, **Pre Requisites**, **Expected Results**, **Pass/Fail Criteria**.
        - Be numbered (e.g., 1., 2.) and separated by two newlines.

        Example:
        1. **Test Scenario**: Invalid input test
        **Test Steps**: 1. Provide invalid input 2. Attempt processing
        **Pre Requisites**: System running
        **Expected Results**: System rejects input with proper error message
        **Pass/Fail Criteria**: Pass if error is correctly displayed

        [/INST]
        """.format(error=error_message, solution=solution, count=required_neg)
        neg_response = llm.invoke(ChatPromptTemplate.from_template(neg_prompt).format_prompt().to_messages()).content.strip()
        generated_neg = [tc.strip() for tc in neg_response.split("\n\n") if tc.strip()]
        generated_neg = generated_neg[:required_neg]

        # Combine generated test cases
        test_cases_text = ""
        for tc in generated_pos:
            test_cases_text += f"**Generated Positive Test Case:**\n{tc}\n\n"
        for tc in generated_neg:
            test_cases_text += f"**Generated Negative Test Case:**\n{tc}\n\n"

        response_template = (
            "**Error:**\n{Error}\n\n"
            "**Solution:**\n{Solution}\n\n"
            "**Explanation:**\n{Explanation}\n\n"
            "**Final Generated Test Cases (2 Positive and 2 Negative):**\n{TestCases}"
        )
        return {"response": response_template.format(
            Error=error_message,
            Solution=solution,
            Explanation=explanation,
            TestCases=test_cases_text
        )}

    except Exception as e:
        logging.error("Validation/Generation error: %s", str(e))
        return {"response": f"Error processing request: {str(e)}"}

# --------------------------------------------------------------------
# Build the State Graph Workflow
# --------------------------------------------------------------------
workflow = StateGraph(AgentState)
# "retrieve" node: fetch context using the input defect
workflow.add_node("retrieve", lambda state: {"context": retriever.invoke(state["input"])})
workflow.add_node("validate_or_generate_test_cases", validate_or_generate_test_cases)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "validate_or_generate_test_cases")
workflow.add_edge("validate_or_generate_test_cases", END)
agent = workflow.compile()

# --------------------------------------------------------------------
# Automated Evaluation & Self-improvement Functions (unchanged)
# --------------------------------------------------------------------
def auto_evaluate_solution(response: str) -> int:
    # Heuristic evaluation: if response contains "Generated", assume good quality.
    if "Generated" in response:
        return 5
    elif "**Error**:" in response:
        return 1
    else:
        return 3

def generate_alternative_solution(error_message: str) -> str:
    alt_prompt = """
    [INST] Provide a concise, actionable alternative solution for the following error:
    Error: {error}
    Ensure that the solution is clear and does not include any follow-up questions.
    [/INST]
    """
    alt_template = ChatPromptTemplate.from_template(alt_prompt)
    formatted_alt = alt_template.format_prompt(error=error_message)
    alternative_solution = llm.invoke(formatted_alt.to_messages()).content.strip()

    test_case_prompt = """
    [INST] Given the error and the alternative solution:
    Error: {error}
    Solution: {solution}
    Generate EXACTLY 4 structured test cases (2 positive and 2 negative) to validate that the alternative solution fixes the issue.
    Each test case must include:
      - **Test Scenario**
      - **Test Steps**
      - **Pre Requisites**
      - **Expected Results**
      - **Pass/Fail Criteria**
    [/INST]
    """
    tc_template = ChatPromptTemplate.from_template(test_case_prompt)
    formatted_tc = tc_template.format_prompt(error=error_message, solution=alternative_solution)
    alternative_test_cases = llm.invoke(formatted_tc.to_messages()).content.strip()

    alt_response = (
        "**Alternative Solution (Generated):**\n{AltSolution}\n\n"
        "**Test Cases for Alternative Solution:**\n{AltTestCases}"
    ).format(
        AltSolution=alternative_solution,
        AltTestCases=alternative_test_cases
    )
    return alt_response

def get_solution_autonomously(error_message: str) -> str:
    max_iterations = 3
    iteration = 0
    while iteration < max_iterations:
        logging.info("Iteration %d: Processing error: %s", iteration + 1, error_message)
        result = agent.invoke({"input": error_message.strip()})
        response = result["response"]
        logging.info("Agent response:\n%s", response)
        rating = auto_evaluate_solution(response)
        logging.info("Auto-evaluated rating: %d", rating)
        if rating < 3:
            logging.info("Rating below threshold. Generating alternative solution.")
            alt_response = generate_alternative_solution(error_message)
            logging.info("Alternative response generated.")
            return alt_response
        else:
            return response
        iteration += 1
    logging.info("Max iterations reached. Returning last response.")
    return response

# --------------------------------------------------------------------
# Autonomous Agent Execution
# --------------------------------------------------------------------
def main():
    error_description = "Search results not displaying correctly"
    final_solution = get_solution_autonomously(error_description)
    print("\n=== Final Autonomous Response ===\n")
    print(final_solution)

if __name__ == "__main__":
    main()



=== Final Autonomous Response ===

**Error:**
Search results not displaying correctly

**Solution:**
Updated Elasticsearch query logic to handle special characters.

**Explanation:**
The error "Search results not displaying correctly" often arises when Elasticsearch, a powerful search engine, struggles to interpret search queries containing special characters. 

Here's why the solution "Updated Elasticsearch query logic to handle special characters" fixes the problem:

* **Special characters as wildcards:**  Many special characters (like *, ?, !, $, etc.) have special meanings in regular expressions, which Elasticsearch uses for querying.  They can act as wildcards, unintentionally expanding your search beyond your intended scope.

* **Encoding issues:**  Special characters might be encoded differently depending on the system or data source. This can lead to Elasticsearch misinterpreting them, resulting in inaccurate or incomplete search results.

**The fix:**

Updating the Elasticsea

In [8]:
#ye wala emergency k liye h


# ... (previous imports and setup remain the same)

# --------------------------------------------------------------------
# Workflow Node: Validate or Generate Test Cases (Enhanced to force generation with delimiters)
# --------------------------------------------------------------------
def validate_or_generate_test_cases(state: AgentState):
    try:
        # Force generation: ignore CSV test cases and always generate 2 positive and 2 negative test cases.
        if not state["context"]:
            return {"response": "**Error**: The defect could not be found in the database."}
        context = state["context"][0]
        error_message = state["input"]
        solution = context.metadata["solution"]
        module = context.metadata["module"]

        # Generate explanation for the solution
        explanation_prompt = """
        [INST] Explain why this solution fixes the following error:
        Error: {error}
        Solution: {solution}
        [/INST]
        """
        explanation_template = ChatPromptTemplate.from_template(explanation_prompt)
        formatted_explanation = explanation_template.format_prompt(error=error_message, solution=solution)
        explanation = llm.invoke(formatted_explanation.to_messages()).content.strip()

        # Define a delimiter that the LLM should include between test cases.
        delimiter = "\n### END TEST CASE ###\n"

        # Generate positive test cases
        pos_prompt = """
        [INST] Generate EXACTLY 2 POSITIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST include the following sections, and end with the delimiter "### END TEST CASE ###":
        - **Test Scenario**: A short description of the scenario.
        - **Test Steps**: Step-by-step instructions.
        - **Pre Requisites**: Conditions before running the test.
        - **Expected Results**: What should happen if the solution works.
        - **Pass/Fail Criteria**: How to determine if the test passes.

        Output format (including the delimiter):
        1. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###

        2. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###
        [/INST]
        """.format(error=error_message, solution=solution)
        pos_template = ChatPromptTemplate.from_template(pos_prompt)
        formatted_pos = pos_template.format_prompt().to_messages()
        pos_response = llm.invoke(formatted_pos).content.strip()
        # Use regex to split by our explicit delimiter
        import re
        generated_pos = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", pos_response) if tc.strip()]
        generated_pos = generated_pos[:2]  # Ensure exactly 2 positive cases

        # Generate negative test cases
        neg_prompt = """
        [INST] Generate EXACTLY 2 NEGATIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST include the following sections, and end with the delimiter "### END TEST CASE ###":
        - **Test Scenario**: A short description of the scenario.
        - **Test Steps**: Step-by-step instructions.
        - **Pre Requisites**: Conditions before running the test.
        - **Expected Results**: What should happen if the solution fails.
        - **Pass/Fail Criteria**: How to determine if the test fails.

        Output format (including the delimiter):
        1. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###

        2. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###
        [/INST]
        """.format(error=error_message, solution=solution)
        neg_template = ChatPromptTemplate.from_template(neg_prompt)
        formatted_neg = neg_template.format_prompt().to_messages()
        neg_response = llm.invoke(formatted_neg).content.strip()
        generated_neg = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", neg_response) if tc.strip()]
        generated_neg = generated_neg[:2]  # Ensure exactly 2 negative cases

        # Combine generated test cases
        test_cases_text = ""
        for idx, tc in enumerate(generated_pos, start=1):
            test_cases_text += f"**Generated Positive Test Case {idx}:**\n{tc}\n\n"
        for idx, tc in enumerate(generated_neg, start=1):
            test_cases_text += f"**Generated Negative Test Case {idx}:**\n{tc}\n\n"

        response_template = (
            "**Error:**\n{Error}\n\n"
            "**Solution:**\n{Solution}\n\n"
            "**Explanation:**\n{Explanation}\n\n"
            "**Final Generated Test Cases (2 Positive and 2 Negative):**\n{TestCases}"
        )
        return {"response": response_template.format(
            Error=error_message,
            Solution=solution,
            Explanation=explanation,
            TestCases=test_cases_text
        )}

    except Exception as e:
        logging.error("Validation/Generation error: %s", str(e))
        return {"response": f"Error processing request: {str(e)}"}

# --------------------------------------------------------------------
# Build the State Graph Workflow
# --------------------------------------------------------------------
workflow = StateGraph(AgentState)
# "retrieve" node: fetch context using the input defect
workflow.add_node("retrieve", lambda state: {"context": retriever.invoke(state["input"])})
workflow.add_node("validate_or_generate_test_cases", validate_or_generate_test_cases)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "validate_or_generate_test_cases")
workflow.add_edge("validate_or_generate_test_cases", END)
agent = workflow.compile()

# --------------------------------------------------------------------
# Automated Evaluation & Self-improvement Functions (unchanged)
# --------------------------------------------------------------------
def auto_evaluate_solution(response: str) -> int:
    # Simple heuristic: if response contains our delimiter, assume generated output is complete.
    if "### END TEST CASE ###" in response:
        return 5
    elif "**Error**:" in response:
        return 1
    else:
        return 3

def generate_alternative_solution(error_message: str) -> str:
    alt_prompt = """
    [INST] Provide a concise, actionable alternative solution for the following error:
    Error: {error}
    Ensure that the solution is clear and does not include any follow-up questions.
    [/INST]
    """
    alt_template = ChatPromptTemplate.from_template(alt_prompt)
    formatted_alt = alt_template.format_prompt(error=error_message)
    alternative_solution = llm.invoke(formatted_alt.to_messages()).content.strip()

    test_case_prompt = """
    [INST] Given the error and the alternative solution:
    Error: {error}
    Solution: {solution}
    Generate EXACTLY 4 structured test cases (2 positive and 2 negative) with the delimiter "### END TEST CASE ###" after each test case.
    Each test case must include:
      - **Test Scenario**
      - **Test Steps**
      - **Pre Requisites**
      - **Expected Results**
      - **Pass/Fail Criteria**
    [/INST]
    """
    tc_template = ChatPromptTemplate.from_template(test_case_prompt)
    formatted_tc = tc_template.format_prompt(error=error_message, solution=alternative_solution)
    alternative_test_cases = llm.invoke(formatted_tc.to_messages()).content.strip()

    alt_response = (
        "**Alternative Solution (Generated):**\n{AltSolution}\n\n"
        "**Test Cases for Alternative Solution:**\n{AltTestCases}"
    ).format(
        AltSolution=alternative_solution,
        AltTestCases=alternative_test_cases
    )
    return alt_response

def get_solution_autonomously(error_message: str) -> str:
    max_iterations = 3
    iteration = 0
    while iteration < max_iterations:
        logging.info("Iteration %d: Processing error: %s", iteration + 1, error_message)
        result = agent.invoke({"input": error_message.strip()})
        response = result["response"]
        logging.info("Agent response:\n%s", response)
        rating = auto_evaluate_solution(response)
        logging.info("Auto-evaluated rating: %d", rating)
        if rating < 3:
            logging.info("Rating below threshold. Generating alternative solution.")
            alt_response = generate_alternative_solution(error_message)
            logging.info("Alternative response generated.")
            return alt_response
        else:
            return response
        iteration += 1
    logging.info("Max iterations reached. Returning last response.")
    return response

# --------------------------------------------------------------------
# Autonomous Agent Execution
# --------------------------------------------------------------------
def main():
    error_description = "Search results not displaying correctly"
    final_solution = get_solution_autonomously(error_description)
    print("\n=== Final Autonomous Response ===\n")
    print(final_solution)

if __name__ == "__main__":
    main()



=== Final Autonomous Response ===

**Error:**
Search results not displaying correctly

**Solution:**
Updated Elasticsearch query logic to handle special characters.

**Explanation:**
The solution fixes the "Search results not displaying correctly" error by addressing a potential issue with how Elasticsearch handles special characters in search queries. 

Here's a breakdown:

* **Special Characters:**  Characters like spaces, punctuation marks, symbols, and accented letters can sometimes be interpreted differently by search engines. This can lead to unexpected search results or no results at all.
* **Elasticsearch Query Logic:** Elasticsearch uses a specific syntax for constructing search queries. If this syntax isn't properly handling special characters, it might misinterpret them, leading to inaccurate searches.
* **The Fix:**  Updating the Elasticsearch query logic to handle special characters means that the system is now better equipped to understand and process these characters co

In [6]:
    error_description = "user is not getting order confirmation email"
    final_solution = get_solution_autonomously(error_description)
    print("\n=== Final Autonomous Response ===\n")
    print(final_solution)


=== Final Autonomous Response ===

**Error:**
user is not getting order confirmation email

**Solution (CSV):**
Corrected SMTP configuration and ensured email queue processing.

**Explanation:**
The solution fixes the "user is not getting order confirmation email" error by addressing two key components:

* **SMTP Configuration:** SMTP (Simple Mail Transfer Protocol) is the standard protocol for sending emails. Incorrect SMTP configuration can prevent emails from being sent at all. By correcting the configuration, the system can now properly connect to the email server and send emails.

* **Email Queue Processing:**  Many applications use an email queue to store emails that need to be sent. If the queue processing is not working correctly, emails may get stuck in the queue and never be delivered. Ensuring the email queue is processing properly means the system can retrieve emails from the queue and send them to the recipients.


Therefore, by fixing both the SMTP configuration and the 

In [1]:
!pip install langgraph langchain pandas faiss-cpu langchain-groq sentence-transformers langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [3]:
# !pip install langgraph langchain pandas faiss-cpu langchain-groq sentence-transformers langchain_community
from google.colab import userdata
groq_api = userdata.get("groq_api_key")

import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from typing import TypedDict, List, Optional
import textwrap

# Load data
defects_df = pd.read_csv("/content/defects.csv")
test_cases_df = pd.read_csv("/content/test_cases.csv")

# Prepare vector stores
def prepare_vector_store(df, content_col, metadata_cols):
    docs = []
    for _, row in df.iterrows():
        if pd.notna(row[content_col]):
            metadata = {col: row[col] for col in metadata_cols if col in row}
            docs.append(Document(page_content=row[content_col], metadata=metadata))
    return FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

# Create vector stores
defects_store = prepare_vector_store(
    defects_df,
    "Description",
    ["Module", "Solution", "Severity_Level", "Affected_Area"]
)

test_cases_store = prepare_vector_store(
    test_cases_df,
    "Test_Scenario",
    ["Module", "Test_Steps", "Pre_Requisite", "Pass_Fail_Criteria", "Expected_Result"]
)

class AgentState(TypedDict):
    input: str
    defect_context: Optional[List[Document]]
    test_case_context: Optional[List[Document]]
    solution: Optional[str]
    explanation: Optional[str]
    generated_test_cases: Optional[str]
    response: Optional[str]

llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.2,
    model_name="mixtral-8x7b-32768"
)

def retrieve_defect(state: AgentState):
    try:
        docs = defects_store.as_retriever(search_kwargs={"k": 1}).invoke(state["input"])
        return {"defect_context": docs}
    except Exception as e:
        print(f"Retrieval error: {e}")
        return {"defect_context": []}

def retrieve_test_cases(state: AgentState):
    try:
        module = state["defect_context"][0].metadata["Module"]
        docs = test_cases_store.as_retriever(search_kwargs={"k": 3}).invoke(
            f"{module} {state['defect_context'][0].metadata['Solution']}"
        )
        return {"test_case_context": docs}
    except Exception as e:
        print(f"Test case retrieval error: {e}")
        return {"test_case_context": []}

def evaluate_test_cases(state: AgentState):
    prompt = ChatPromptTemplate.from_template("""
    [INST] Evaluate if these test cases adequately validate the solution:
    Solution: {solution}
    Test Cases: {test_cases}
    Answer only 'yes' or 'no'. [/INST]
    """)

    test_cases = "\n".join([doc.page_content for doc in state["test_case_context"]])
    evaluation = llm.invoke(prompt.format(
        solution=state["defect_context"][0].metadata["Solution"],
        test_cases=test_cases
    )).content.strip().lower()

    # If evaluation returns "yes", keep CSV test cases; otherwise clear them to trigger generation.
    return {"test_case_context": state["test_case_context"]} if evaluation == "yes" else {"test_case_context": []}

def generate_solution(state: AgentState):
    prompt = ChatPromptTemplate.from_template("""
    [INST] Generate a solution for this defect:
    {defect}
    Provide the solution and a brief explanation separated by '|'. [/INST]
    """)

    response = llm.invoke(prompt.format(defect=state["input"])).content
    solution, explanation = response.split("|", 1)
    return {"solution": solution.strip(), "explanation": explanation.strip()}

def generate_test_cases(state: AgentState):
    solution = state.get("solution") or state["defect_context"][0].metadata["Solution"]

    prompt = ChatPromptTemplate.from_template("""
    [INST] Generate 3 comprehensive test cases to validate this solution:
    {solution}
    For each test case include:
    - Test Scenario
    - Test Steps
    - Pre-Requisite
    - Pass/Fail Criteria
    - Expected Result
    Format with numbers and bullet points. [/INST]
    """)

    test_cases = llm.invoke(prompt.format(solution=solution)).content
    return {"generated_test_cases": textwrap.dedent(test_cases)}

def format_response(state: AgentState):
    response = []

    # Determine if the solution is sourced from CSV or generated.
    if state.get("defect_context"):
        solution = state["defect_context"][0].metadata["Solution"]
        solution_source = "CSV"
    else:
        solution = state["solution"]
        solution_source = "Generated"

    # Ensure an explanation is available; generate one if not.
    if not state.get("explanation"):
        explanation_prompt = ChatPromptTemplate.from_template("""
        [INST] Provide a brief explanation for the following solution:
        {solution}
        [/INST]
        """)
        explanation = llm.invoke(explanation_prompt.format(solution=solution)).content.strip()
        state["explanation"] = explanation

    response.append(f"**Defect:** {state['input']}")
    response.append(f"**Solution ({solution_source}):** {solution}")
    response.append(f"**Explanation:** {state['explanation']}")

    # Check for test cases and specify the source accordingly.
    if state.get("test_case_context"):
        test_case_source = "CSV"
        response.append("**Validating Test Cases (from CSV):**")
        for i, doc in enumerate(state["test_case_context"], 1):
            response.append(f"{i}. {doc.page_content}")
    else:
        test_case_source = "Generated"
        response.append("**Generated Test Cases:**")
        response.append(state["generated_test_cases"])

    response.append(f"**Test Cases Source:** {test_case_source}")

    # Provide feedback input space and thank the user.
    response.append("\nPlease enter your feedback (a rating from 1-5 and any comments):")
    response.append("Thank you for your feedback and for using our solution!")

    return {"response": "\n\n".join(response)}

workflow = StateGraph(AgentState)

workflow.add_node("retrieve_defect", retrieve_defect)
workflow.add_node("retrieve_test_cases", retrieve_test_cases)
workflow.add_node("evaluate_test_cases", evaluate_test_cases)
workflow.add_node("generate_solution", generate_solution)
workflow.add_node("generate_test_cases", generate_test_cases)
workflow.add_node("format_response", format_response)

workflow.set_entry_point("retrieve_defect")

workflow.add_conditional_edges(
    "retrieve_defect",
    lambda state: bool(state["defect_context"]),
    {
        True: "retrieve_test_cases",
        False: "generate_solution"
    }
)

workflow.add_conditional_edges(
    "retrieve_test_cases",
    lambda state: bool(state["test_case_context"]),
    {
        True: "evaluate_test_cases",
        False: "generate_test_cases"
    }
)

workflow.add_conditional_edges(
    "evaluate_test_cases",
    lambda state: bool(state["test_case_context"]),
    {
        True: "format_response",
        False: "generate_test_cases"
    }
)

workflow.add_edge("generate_solution", "generate_test_cases")
workflow.add_edge("generate_test_cases", "format_response")
workflow.add_edge("format_response", END)

agent = workflow.compile()

# Example usage
def resolve_defect(defect_description):
    result = agent.invoke({"input": defect_description})
    return result["response"]

# Test the agent
print(resolve_defect("Payment gateway timeout during checkout"))


<ipython-input-3-aa2c75bab8a3>:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acc

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Defect:** Payment gateway timeout during checkout

**Solution (CSV):** Optimized API calls to payment service provider and added retry logic.

**Explanation:** Sure, I'd be happy to explain!

An API (Application Programming Interface) call is a method used by software applications to communicate with each other. In this case, the API calls are being made to a payment service provider.

Optimizing API calls means making them as efficient as possible. This can be done by reducing the number of calls made, minimizing the amount of data transferred, and improving the response time of the calls. By optimizing API calls, the overall performance of the system is improved, resulting in faster and more reliable payment processing.

Adding retry logic means that if a payment processing attempt fails, the system will automatically retry the process a certain number of times before giving up. This is important because sometimes payment processing can fail due to temporary issues, such as network

In [5]:
# !pip install langgraph langchain pandas faiss-cpu langchain-groq sentence-transformers langchain_community
from google.colab import userdata
groq_api = userdata.get("groq_api_key")

import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from typing import TypedDict, List, Optional
import textwrap

# Load data
defects_df = pd.read_csv("/content/defects.csv")
test_cases_df = pd.read_csv("/content/test_cases.csv")

# Prepare vector stores
def prepare_vector_store(df, content_col, metadata_cols):
    docs = []
    for _, row in df.iterrows():
        if pd.notna(row[content_col]):
            metadata = {col: row[col] for col in metadata_cols if col in row}
            docs.append(Document(page_content=row[content_col], metadata=metadata))
    return FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

# Create vector stores for defects and test cases
defects_store = prepare_vector_store(
    defects_df,
    "Description",
    ["Module", "Solution", "Severity_Level", "Affected_Area"]
)

test_cases_store = prepare_vector_store(
    test_cases_df,
    "Test_Scenario",
    ["Module", "Test_Steps", "Pre_Requisite", "Pass_Fail_Criteria", "Expected_Result"]
)

class AgentState(TypedDict):
    input: str
    defect_context: Optional[List[Document]]
    test_case_context: Optional[List[Document]]
    solution: Optional[str]
    explanation: Optional[str]
    generated_test_cases: Optional[str]
    response: Optional[str]

# Initialize the LLM
llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.2,
    model_name="mixtral-8x7b-32768"
)

def retrieve_defect(state: AgentState):
    try:
        docs = defects_store.as_retriever(search_kwargs={"k": 1}).invoke(state["input"])
        return {"defect_context": docs}
    except Exception as e:
        print(f"Retrieval error: {e}")
        return {"defect_context": []}

def retrieve_test_cases(state: AgentState):
    try:
        module = state["defect_context"][0].metadata["Module"]
        docs = test_cases_store.as_retriever(search_kwargs={"k": 3}).invoke(
            f"{module} {state['defect_context'][0].metadata['Solution']}"
        )
        return {"test_case_context": docs}
    except Exception as e:
        print(f"Test case retrieval error: {e}")
        return {"test_case_context": []}

def evaluate_test_cases(state: AgentState):
    prompt = ChatPromptTemplate.from_template("""
    [INST] Evaluate if these test cases adequately validate the solution:
    Solution: {solution}
    Test Cases: {test_cases}
    Answer only 'yes' or 'no'. [/INST]
    """)

    test_cases = "\n".join([doc.page_content for doc in state["test_case_context"]])
    evaluation = llm.invoke(prompt.format(
        solution=state["defect_context"][0].metadata["Solution"],
        test_cases=test_cases
    )).content.strip().lower()

    # If evaluation returns "yes", keep CSV test cases; otherwise clear them to trigger generation.
    return {"test_case_context": state["test_case_context"]} if evaluation == "yes" else {"test_case_context": []}

def generate_solution(state: AgentState):
    prompt = ChatPromptTemplate.from_template("""
    [INST] Generate a solution for this defect:
    {defect}
    Provide the solution and a brief explanation separated by '|'. [/INST]
    """)

    response = llm.invoke(prompt.format(defect=state["input"])).content
    solution, explanation = response.split("|", 1)
    return {"solution": solution.strip(), "explanation": explanation.strip()}

def generate_test_cases(state: AgentState):
    solution = state.get("solution") or state["defect_context"][0].metadata["Solution"]

    prompt = ChatPromptTemplate.from_template("""
    [INST] Generate 3 comprehensive test cases to validate this solution:
    {solution}
    For each test case include:
    - Test Scenario
    - Test Steps
    - Pre-Requisite
    - Pass/Fail Criteria
    - Expected Result
    Format with numbers and bullet points. [/INST]
    """)

    test_cases = llm.invoke(prompt.format(solution=solution)).content
    return {"generated_test_cases": textwrap.dedent(test_cases)}

def format_response(state: AgentState):
    response = []

    # Determine if the solution is sourced from CSV or generated.
    if state.get("defect_context"):
        solution = state["defect_context"][0].metadata["Solution"]
        solution_source = "CSV"
    else:
        solution = state["solution"]
        solution_source = "Generated"

    # Ensure an explanation is available; generate one if not.
    if not state.get("explanation"):
        explanation_prompt = ChatPromptTemplate.from_template("""
        [INST] Provide a brief explanation for the following solution:
        {solution}
        [/INST]
        """)
        explanation = llm.invoke(explanation_prompt.format(solution=solution)).content.strip()
        state["explanation"] = explanation

    response.append(f"**Defect:** {state['input']}")
    response.append(f"**Solution ({solution_source}):** {solution}")
    response.append(f"**Explanation:** {state['explanation']}")

    # Check for test cases and specify the source accordingly.
    if state.get("test_case_context"):
        test_case_source = "CSV"
        response.append("**Validating Test Cases (from CSV):**")
        for i, doc in enumerate(state["test_case_context"], 1):
            response.append(f"{i}. {doc.page_content}")
    else:
        test_case_source = "Generated"
        response.append("**Generated Test Cases:**")
        response.append(state["generated_test_cases"])

    response.append(f"**Test Cases Source:** {test_case_source}")

    # Provide a prompt for feedback (this will be printed to the user)
    response.append("\nPlease enter your feedback (a rating from 1-5 and any comments):")

    return {"response": "\n\n".join(response)}

# Define the workflow and nodes
workflow = StateGraph(AgentState)

workflow.add_node("retrieve_defect", retrieve_defect)
workflow.add_node("retrieve_test_cases", retrieve_test_cases)
workflow.add_node("evaluate_test_cases", evaluate_test_cases)
workflow.add_node("generate_solution", generate_solution)
workflow.add_node("generate_test_cases", generate_test_cases)
workflow.add_node("format_response", format_response)

workflow.set_entry_point("retrieve_defect")

workflow.add_conditional_edges(
    "retrieve_defect",
    lambda state: bool(state["defect_context"]),
    {
        True: "retrieve_test_cases",
        False: "generate_solution"
    }
)

workflow.add_conditional_edges(
    "retrieve_test_cases",
    lambda state: bool(state["test_case_context"]),
    {
        True: "evaluate_test_cases",
        False: "generate_test_cases"
    }
)

workflow.add_conditional_edges(
    "evaluate_test_cases",
    lambda state: bool(state["test_case_context"]),
    {
        True: "format_response",
        False: "generate_test_cases"
    }
)

workflow.add_edge("generate_solution", "generate_test_cases")
workflow.add_edge("generate_test_cases", "format_response")
workflow.add_edge("format_response", END)

agent = workflow.compile()

# Enhanced usage function with interactive feedback collection
def resolve_defect(defect_description):
    result = agent.invoke({"input": defect_description})
    # Print the agent's response (solution, explanation, test cases)
    print(result["response"])

    # Prompt the user for feedback
    feedback = input("\nEnter your feedback (a rating from 1-5 and any comments): ")
    print("\nThank you for your feedback!")
    # Optionally, you can process or store the feedback here.

    return result["response"]

# Test the agent
print(resolve_defect("Login button not responding"))


**Defect:** Login button not responding

**Solution (CSV):** Fixed button click event listener.

**Explanation:** A fixed button click event listener is a programming construct that allows a specific action to be triggered when a button is clicked, while keeping the button in a fixed position on the screen.

Here's a brief explanation:

1. **Fixed Button**: A button is an HTML element that represents a clickable control. When a button is fixed, it stays in the same position on the screen even when the user scrolls. This is achieved using CSS position property with a value of 'fixed'.

2. **Event Listener**: An event listener is a JavaScript function that waits for an event to occur, such as a button click. When the event occurs, the function is executed.

3. **Button Click Event**: The button click event is triggered when a user clicks on a button. This event can be captured and used to perform a specific action.

4. **Fixed Button Click Event Listener**: A fixed button click event lis

In [7]:
print(resolve_defect("Search results not displaying

KeyboardInterrupt: Interrupted by user

In [2]:
from google.colab import userdata
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import END, StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from typing import TypedDict, List

# Load Groq API key and CSV files
groq_api = userdata.get("groq_api_key")
df = pd.read_csv("/content/defects.csv")            # Contains defects and solutions
test_cases_df = pd.read_csv("/content/test_cases.csv")  # Contains test cases

# Prepare documents for retrieval
docs = []
for _, row in df.iterrows():
    if pd.notna(row["Description"]) and pd.notna(row["Solution"]):
        docs.append(Document(
            page_content=row["Description"],
            metadata={
                "solution": row["Solution"],
                "module": row["Module"]
            }
        ))

# Create vector store using embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# Define state for the agent
class AgentState(TypedDict):
    input: str
    context: List[Document]
    response: str

# Initialize the LLM
llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.3,
    model_name="gemma2-9b-it",
)

# Step 1: Retrieve the solution from CSV based on the defect description
def retrieve(state: AgentState):
    try:
        relevant_docs = retriever.invoke(state["input"])
        return {"context": relevant_docs}
    except Exception as e:
        return {"context": []}

# Step 2: Validate solution with test cases (from CSV) or generate new ones if not found.
def validate_or_generate_test_cases(state: AgentState):
    try:
        if not state["context"]:
            return {"response": "**Error**: The defect could not be found in the database."}

        context = state["context"][0]
        error_message = state["input"]
        solution = context.metadata["solution"]
        module = context.metadata["module"]

        # Generate an explanation for the solution.
        explanation_prompt = """
        [INST] Explain why this solution fixes the following error:
        Error: {error}
        Solution: {solution}
        [/INST]
        """
        explanation = llm.invoke(ChatPromptTemplate.from_template(explanation_prompt).format(
            error=error_message,
            solution=solution
        )).content.strip()

        # Check if test cases exist for the module.
        matching_test_cases = test_cases_df[test_cases_df["Module"] == module]
        if not matching_test_cases.empty:
            test_cases = "\n".join([
                f"**Test Scenario (Fetched from CSV)**: {row['Test_Scenario']}\n"
                f"**Test Steps**: {row['Test_Steps']}\n"
                f"**Pre Requisites**: {row['Pre_Requisite']}\n"
                f"**Expected Result**: {row['Expected_Result']}\n"
                f"**Pass/Fail Criteria**: {row['Pass_Fail_Criteria']}\n"
                for _, row in matching_test_cases.iterrows()
            ])
            response_template = (
                "**Error:**\n{Error}\n\n"
                "**Solution (Fetched from CSV):**\n{Solution}\n\n"
                "**Explanation:**\n{Explanation}\n\n"
                "**Test Cases (Fetched from CSV):**\n{TestCases}"
            )
            return {"response": response_template.format(
                Error=error_message,
                Solution=solution,
                Explanation=explanation,
                TestCases=test_cases
            )}

        # Otherwise, generate new test cases using the LLM.
        prompt_template = """
        [INST] Given this error and known solution:
        Error: {error}
        Solution: {solution}
        Generate **exactly** 2 structured test cases to validate that the solution fixes the issue:
        - 1 Positive Test Case (where the solution works correctly)
        - 1 Negative Test Case (where the solution fails or is misconfigured)
        Each test case should be unique and structured in the following format:
          1. **Test Scenario**: Description of what is being tested
          2. **Test Steps**: Step-by-step actions to perform
          3. **Pre Requisites**: What must be in place before testing
          4. **Expected Results**: Expected outcomes if the solution works
          5. **Pass/Fail Criteria**: How to determine if the test passes or fails
        [/INST]
        """
        generated_test_cases = llm.invoke(ChatPromptTemplate.from_template(prompt_template).format(
            error=error_message,
            solution=solution
        )).content.strip()
        response_template = (
            "**Error:**\n{Error}\n\n"
            "**Solution (Fetched from CSV):**\n{Solution}\n\n"
            "**Explanation:**\n{Explanation}\n\n"
            "**Generated Test Cases (Generated by Agent):**\n{TestCases}"
        )
        return {"response": response_template.format(
            Error=error_message,
            Solution=solution,
            Explanation=explanation,
            TestCases=generated_test_cases
        )}
    except Exception as e:
        return {"response": f"Error processing request: {str(e)}"}

# Define the workflow of the agent using a state graph.
workflow = StateGraph(AgentState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("validate_or_generate_test_cases", validate_or_generate_test_cases)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "validate_or_generate_test_cases")
workflow.add_edge("validate_or_generate_test_cases", END)
agent = workflow.compile()

# Feedback storage: set up an empty DataFrame to store feedback.
feedback_df = pd.DataFrame(columns=["Error", "Solution", "Rating"])

def collect_feedback(error_message, solution, rating):
    global feedback_df
    new_feedback = pd.DataFrame({
        "Error": [error_message],
        "Solution": [solution],
        "Rating": [rating]
    })
    feedback_df = pd.concat([feedback_df, new_feedback], ignore_index=True)
    feedback_df.to_csv("/content/feedback.csv", index=False)

# Function that runs the agent, collects feedback, and, if necessary, processes a re-generation.
def get_solution_with_feedback(error_message):
    while True:
        # Run the agent to obtain a solution and test cases.
        result = agent.invoke({"input": error_message.strip()})
        print(result["response"])

        # Ask the user to rate the solution.
        try:
            rating = int(input("Rate the solution (1-5): "))
            if rating < 1 or rating > 5:
                print("Invalid rating. Please enter a number between 1 and 5.")
                continue
        except ValueError:
            print("Invalid input. Please enter a valid number between 1 and 5.")
            continue

        # Save the feedback.
        collect_feedback(error_message, result["response"], rating)

        # If the rating is below 3, generate an alternative solution.
        if rating < 3:
            print("\nYour rating indicates that this solution could be improved. Generating an alternative solution...\n")

            alternative_prompt = """
            [INST] Provide a concise and actionable alternative solution for the following error:
            Error: {error}
            Ensure that the solution is clear and does not include any follow-up questions.
            [/INST]
            """
            alternative_solution = llm.invoke(ChatPromptTemplate.from_template(alternative_prompt).format(
                error=error_message
            )).content.strip()

            # Generate test cases for the alternative solution.
            test_case_prompt = """
            [INST] Given the error and the new alternative solution:
            Error: {error}
            Solution: {solution}
            Generate **exactly** 2 structured test cases to validate that this alternative solution fixes the issue:
            - 1 Positive Test Case (solution works)
            - 1 Negative Test Case (solution fails or is misconfigured)
            Each test case must include:
              1. **Test Scenario**
              2. **Test Steps**
              3. **Pre Requisites**
              4. **Expected Results**
              5. **Pass/Fail Criteria**
            [/INST]
            """
            alternative_test_cases = llm.invoke(ChatPromptTemplate.from_template(test_case_prompt).format(
                error=error_message,
                solution=alternative_solution
            )).content.strip()

            print(f"**Alternative Solution (Generated by Agent):**\n{alternative_solution}")
            print(f"\n**Generated Test Cases (Generated by Agent):**\n{alternative_test_cases}")

            # Store feedback for the alternative solution as well.
            collect_feedback(error_message, alternative_solution, rating)
            break  # Exit the feedback loop after generating and displaying the alternative solution.
        else:
            print("Thank you for your feedback!")
            break

# Example usage: Provide the error description and let the agent process the feedback.
if __name__ == "__main__":
    print("=== Exact Error Example ===")
    get_solution_with_feedback("Search results not displaying correctly")


<ipython-input-2-9afa2067490b>:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

=== Exact Error Example ===
**Error:**
Search results not displaying correctly

**Solution (Fetched from CSV):**
Updated Elasticsearch query logic to handle special characters.

**Explanation:**
The solution fixes the "Search results not displaying correctly" error by addressing a potential issue with how Elasticsearch handles special characters in search queries. 

Here's why:

* **Special characters can cause problems:**  Characters like spaces, punctuation marks, or symbols can sometimes be misinterpreted by Elasticsearch during the search process. This can lead to incorrect results, missing results, or unexpected behavior.
* **Updated query logic:** The solution states that the Elasticsearch query logic has been updated. This means the way Elasticsearch interprets and processes search terms has been changed to better handle special characters. 
* **Improved accuracy:** By addressing the special character issue, the updated query logic should ensure that search terms are parsed corr

In [2]:
#9/2/25, rat k 10:30 ho rahe h, or ye wala he code final rakhna h, isi ko explain krna h apn ne

from google.colab import userdata
import pandas as pd
import numpy as np
import logging
import random
import threading
import sys
import time
from typing import TypedDict, List

# Import LangGraph and related components
from langgraph.graph import END, StateGraph
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# Setup logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --------------------------------------------------------------------
# Data Loading and Document Preparation
# --------------------------------------------------------------------
groq_api = userdata.get("groq_api_key")
df = pd.read_csv("/content/defects.csv")             # Contains defects and solutions
test_cases_df = pd.read_csv("/content/test_cases.csv")  # Contains test cases

docs = []
for _, row in df.iterrows():
    if pd.notna(row["Description"]) and pd.notna(row["Solution"]):
        docs.append(Document(
            page_content=row["Description"],
            metadata={
                "solution": row["Solution"],
                "module": row["Module"]
            }
        ))

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# --------------------------------------------------------------------
# Define Agent State and LLM Initialization
# --------------------------------------------------------------------
class AgentState(TypedDict):
    input: str
    context: List[Document]
    response: str

llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.3,
    model_name="gemma2-9b-it",
)

# --------------------------------------------------------------------
# Helper: Classify Test Case (Positive/Negative) with Enhanced Keywords
# --------------------------------------------------------------------
def classify_test_case(tc_text: str) -> str:
    negative_keywords = [
        "fail", "error", "misconfigured", "incorrect", "doesn't work",
        "not work", "negative", "invalid", "wrong", "missing", "unexpected",
        "should not", "incorrectly", "failure", "reject", "malformed",
        "timeout", "invalid input", "edge case", "out of bounds"
    ]
    text_lower = tc_text.lower()
    return "negative" if any(kw in text_lower for kw in negative_keywords) else "positive"

# ... (previous imports and setup remain the same)

# --------------------------------------------------------------------
# Workflow Node: Validate or Generate Test Cases (Enhanced to force generation with delimiters)
# --------------------------------------------------------------------
def validate_or_generate_test_cases(state: AgentState):
    try:
        # Force generation: ignore CSV test cases and always generate 2 positive and 2 negative test cases.
        if not state["context"]:
            return {"response": "**Error**: The defect could not be found in the database."}
        context = state["context"][0]
        error_message = state["input"]
        solution = context.metadata["solution"]
        module = context.metadata["module"]

        # Generate explanation for the solution
        explanation_prompt = """
        [INST] Explain why this solution fixes the following error:
        Error: {error}
        Solution: {solution}
        [/INST]
        """
        explanation_template = ChatPromptTemplate.from_template(explanation_prompt)
        formatted_explanation = explanation_template.format_prompt(error=error_message, solution=solution)
        explanation = llm.invoke(formatted_explanation.to_messages()).content.strip()

        # Define a delimiter that the LLM should include between test cases.
        delimiter = "\n### END TEST CASE ###\n"

        # Generate positive test cases
        pos_prompt = """
        [INST] Generate EXACTLY 2 POSITIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST include the following sections, and end with the delimiter "### END TEST CASE ###":
        - **Test Scenario**: A short description of the scenario.
        - **Test Steps**: Step-by-step instructions.
        - **Pre Requisites**: Conditions before running the test.
        - **Expected Results**: What should happen if the solution works.
        - **Pass/Fail Criteria**: How to determine if the test passes.

        Output format (including the delimiter):
        1. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###

        2. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###
        [/INST]
        """.format(error=error_message, solution=solution)
        pos_template = ChatPromptTemplate.from_template(pos_prompt)
        formatted_pos = pos_template.format_prompt().to_messages()
        pos_response = llm.invoke(formatted_pos).content.strip()
        # Use regex to split by our explicit delimiter
        import re
        generated_pos = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", pos_response) if tc.strip()]
        generated_pos = generated_pos[:2]  # Ensure exactly 2 positive cases

        # Generate negative test cases
        neg_prompt = """
        [INST] Generate EXACTLY 2 NEGATIVE test case(s) for:
        Error: {error}
        Solution: {solution}

        Each test case MUST include the following sections, and end with the delimiter "### END TEST CASE ###":
        - **Test Scenario**: A short description of the scenario.
        - **Test Steps**: Step-by-step instructions.
        - **Pre Requisites**: Conditions before running the test.
        - **Expected Results**: What should happen if the solution fails.
        - **Pass/Fail Criteria**: How to determine if the test fails.

        Output format (including the delimiter):
        1. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###

        2. **Test Scenario**: <scenario details>
           **Test Steps**: <step-by-step details>
           **Pre Requisites**: <pre-requisites>
           **Expected Results**: <expected results>
           **Pass/Fail Criteria**: <criteria>
        ### END TEST CASE ###
        [/INST]
        """.format(error=error_message, solution=solution)
        neg_template = ChatPromptTemplate.from_template(neg_prompt)
        formatted_neg = neg_template.format_prompt().to_messages()
        neg_response = llm.invoke(formatted_neg).content.strip()
        generated_neg = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", neg_response) if tc.strip()]
        generated_neg = generated_neg[:2]  # Ensure exactly 2 negative cases

        # Combine generated test cases
        test_cases_text = ""
        for idx, tc in enumerate(generated_pos, start=1):
            test_cases_text += f"**Generated Positive Test Case {idx}:**\n{tc}\n\n"
        for idx, tc in enumerate(generated_neg, start=1):
            test_cases_text += f"**Generated Negative Test Case {idx}:**\n{tc}\n\n"

        response_template = (
            "**Error:**\n{Error}\n\n"
            "**Solution:**\n{Solution}\n\n"
            "**Explanation:**\n{Explanation}\n\n"
            "**Final Generated Test Cases (2 Positive and 2 Negative):**\n{TestCases}"
        )
        return {"response": response_template.format(
            Error=error_message,
            Solution=solution,
            Explanation=explanation,
            TestCases=test_cases_text
        )}

    except Exception as e:
        logging.error("Validation/Generation error: %s", str(e))
        return {"response": f"Error processing request: {str(e)}"}

# --------------------------------------------------------------------
# Build the State Graph Workflow
# --------------------------------------------------------------------
workflow = StateGraph(AgentState)
# "retrieve" node: fetch context using the input defect
workflow.add_node("retrieve", lambda state: {"context": retriever.invoke(state["input"])})
workflow.add_node("validate_or_generate_test_cases", validate_or_generate_test_cases)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "validate_or_generate_test_cases")
workflow.add_edge("validate_or_generate_test_cases", END)
agent = workflow.compile()

# --------------------------------------------------------------------
# Automated Evaluation & Self-improvement Functions (unchanged)
# --------------------------------------------------------------------
def auto_evaluate_solution(response: str) -> int:
    # Simple heuristic: if response contains our delimiter, assume generated output is complete.
    if "### END TEST CASE ###" in response:
        return 5
    elif "**Error**:" in response:
        return 1
    else:
        return 3

def generate_alternative_solution(error_message: str) -> str:
    alt_prompt = """
    [INST] Provide a concise, actionable alternative solution for the following error:
    Error: {error}
    Ensure that the solution is clear and does not include any follow-up questions.
    [/INST]
    """
    alt_template = ChatPromptTemplate.from_template(alt_prompt)
    formatted_alt = alt_template.format_prompt(error=error_message)
    alternative_solution = llm.invoke(formatted_alt.to_messages()).content.strip()

    test_case_prompt = """
    [INST] Given the error and the alternative solution:
    Error: {error}
    Solution: {solution}
    Generate EXACTLY 4 structured test cases (2 positive and 2 negative) with the delimiter "### END TEST CASE ###" after each test case.
    Each test case must include:
      - **Test Scenario**
      - **Test Steps**
      - **Pre Requisites**
      - **Expected Results**
      - **Pass/Fail Criteria**
    [/INST]
    """
    tc_template = ChatPromptTemplate.from_template(test_case_prompt)
    formatted_tc = tc_template.format_prompt(error=error_message, solution=alternative_solution)
    alternative_test_cases = llm.invoke(formatted_tc.to_messages()).content.strip()

    alt_response = (
        "**Alternative Solution (Generated):**\n{AltSolution}\n\n"
        "**Test Cases for Alternative Solution:**\n{AltTestCases}"
    ).format(
        AltSolution=alternative_solution,
        AltTestCases=alternative_test_cases
    )
    return alt_response

def get_solution_autonomously(error_message: str) -> str:
    max_iterations = 3
    iteration = 0
    while iteration < max_iterations:
        logging.info("Iteration %d: Processing error: %s", iteration + 1, error_message)
        result = agent.invoke({"input": error_message.strip()})
        response = result["response"]
        logging.info("Agent response:\n%s", response)
        rating = auto_evaluate_solution(response)
        logging.info("Auto-evaluated rating: %d", rating)
        if rating < 3:
            logging.info("Rating below threshold. Generating alternative solution.")
            alt_response = generate_alternative_solution(error_message)
            logging.info("Alternative response generated.")
            return alt_response
        else:
            return response
        iteration += 1
    logging.info("Max iterations reached. Returning last response.")
    return response

# --------------------------------------------------------------------
# Autonomous Agent Execution
# --------------------------------------------------------------------
def main():
    error_description = "Search results not displaying correctly"
    final_solution = get_solution_autonomously(error_description)
    print("\n=== Final Autonomous Response ===\n")
    print(final_solution)

if __name__ == "__main__":
    main()


<ipython-input-2-b44a3755ed56>:40: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== Final Autonomous Response ===

**Error:**
Search results not displaying correctly

**Solution:**
Updated Elasticsearch query logic to handle special characters.

**Explanation:**
The error "Search results not displaying correctly" often points to issues with how the search engine (in this case, Elasticsearch) is interpreting search queries.  

Here's why updating the Elasticsearch query logic to handle special characters fixes this problem:

* **Special characters can be misinterpreted:**  Characters like spaces, punctuation marks, symbols, and accented letters can sometimes be treated as delimiters or have unexpected meanings within a search query. This can lead to the search engine returning incorrect or incomplete results.

* **Query logic handles special characters:**  Elasticsearch query logic defines how the search engine processes these characters. By updating this logic, you can ensure that:
    * **Special characters are treated as literal text:**  Instead of being interp

In [3]:
from google.colab import userdata
import pandas as pd
import numpy as np
import logging
import random
import re
import sys
import time
from datetime import timedelta, datetime
from typing import TypedDict, List

# Import LangGraph and related components
from langgraph.graph import END, StateGraph
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

# Setup logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --------------------------------------------------------------------
# Data Loading and Document Preparation
# --------------------------------------------------------------------
groq_api = userdata.get("groq_api_key")
df = pd.read_csv("/content/defects.csv")             # Contains defects and solutions
test_cases_df = pd.read_csv("/content/test_cases.csv")  # Contains test cases

docs = []
for _, row in df.iterrows():
    if pd.notna(row["Description"]) and pd.notna(row["Solution"]):
        docs.append(Document(
            page_content=row["Description"],
            metadata={
                "solution": row["Solution"],
                "module": row["Module"]
            }
        ))

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# --------------------------------------------------------------------
# Define Agent State and LLM Initialization
# --------------------------------------------------------------------
class AgentState(TypedDict):
    input: str
    context: List[Document]
    response: str

llm = ChatGroq(
    groq_api_key=groq_api,
    temperature=0.3,
    model_name="gemma2-9b-it",
)

# --------------------------------------------------------------------
# Helper: Classify Test Case (Positive/Negative) with Enhanced Keywords
# --------------------------------------------------------------------
def classify_test_case(tc_text: str) -> str:
    negative_keywords = [
        "fail", "error", "misconfigured", "incorrect", "doesn't work",
        "not work", "negative", "invalid", "wrong", "missing", "unexpected",
        "should not", "incorrectly", "failure", "reject", "malformed",
        "timeout", "invalid input", "edge case", "out of bounds"
    ]
    text_lower = tc_text.lower()
    return "negative" if any(kw in text_lower for kw in negative_keywords) else "positive"

# --------------------------------------------------------------------
# Workflow Node: Validate or Generate Test Cases (Enhanced with source labels)
# --------------------------------------------------------------------
def validate_or_generate_test_cases(state: AgentState):
    try:
        if not state["context"]:
            return {"response": "**Error**: The defect could not be found in the database."}
        context = state["context"][0]
        error_message = state["input"]
        solution = context.metadata["solution"]
        module = context.metadata["module"]

        # Generate explanation for the solution
        explanation_prompt = """
        [INST] Explain why this solution fixes the following error:
        Error: {error}
        Solution: {solution}
        [/INST]
        """
        explanation_template = ChatPromptTemplate.from_template(explanation_prompt)
        formatted_explanation = explanation_template.format_prompt(error=error_message, solution=solution)
        explanation = llm.invoke(formatted_explanation.to_messages()).content.strip()

        # --- Process CSV Test Cases ---
        csv_test_cases_pos = []
        csv_test_cases_neg = []
        matching_test_cases = test_cases_df[test_cases_df["Module"] == module]
        if not matching_test_cases.empty:
            for _, row in matching_test_cases.iterrows():
                tc = (
                    f"**Test Scenario:** {row['Test_Scenario']}\n"
                    f"**Test Steps:** {row['Test_Steps']}\n"
                    f"**Pre Requisites:** {row['Pre_Requisite']}\n"
                    f"**Expected Result:** {row['Expected_Result']}\n"
                    f"**Pass/Fail Criteria:** {row['Pass_Fail_Criteria']}\n"
                )
                if classify_test_case(tc) == "positive":
                    csv_test_cases_pos.append(f"**CSV Positive Test Case:**\n{tc}")
                else:
                    csv_test_cases_neg.append(f"**CSV Negative Test Case:**\n{tc}")

        # Determine how many generated cases are needed per type
        required_pos = 2
        required_neg = 2
        missing_pos = max(required_pos - len(csv_test_cases_pos), 0)
        missing_neg = max(required_neg - len(csv_test_cases_neg), 0)

        # Use a delimiter for splitting generated outputs
        delimiter = "\n### END TEST CASE ###\n"

        generated_pos = []
        if missing_pos > 0:
            pos_prompt = """
            [INST] Generate EXACTLY {count} POSITIVE test case(s) for:
            Error: {error}
            Solution: {solution}

            Each test case MUST include:
            - **Test Scenario**: A short description of the scenario.
            - **Test Steps**: Step-by-step instructions.
            - **Pre Requisites**: Conditions before running the test.
            - **Expected Results**: What should happen if the solution works.
            - **Pass/Fail Criteria**: How to determine if the test passes.

            Append the delimiter "### END TEST CASE ###" after each test case.

            Format:
            1. **Test Scenario**: <scenario details>
               **Test Steps**: <steps>
               **Pre Requisites**: <conditions>
               **Expected Results**: <expected results>
               **Pass/Fail Criteria**: <criteria>
            ### END TEST CASE ###
            [/INST]
            """.format(error=error_message, solution=solution, count=missing_pos)
            pos_template = ChatPromptTemplate.from_template(pos_prompt)
            formatted_pos = pos_template.format_prompt().to_messages()
            pos_response = llm.invoke(formatted_pos).content.strip()
            generated_pos = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", pos_response) if tc.strip()]
            generated_pos = [f"**Generated Positive Test Case:**\n{tc}" for tc in generated_pos][:missing_pos]

        generated_neg = []
        if missing_neg > 0:
            neg_prompt = """
            [INST] Generate EXACTLY {count} NEGATIVE test case(s) for:
            Error: {error}
            Solution: {solution}

            Each test case MUST include:
            - **Test Scenario**: A short description of the scenario.
            - **Test Steps**: Step-by-step instructions.
            - **Pre Requisites**: Conditions before running the test.
            - **Expected Results**: What should happen if the solution fails.
            - **Pass/Fail Criteria**: How to determine if the test fails.

            Append the delimiter "### END TEST CASE ###" after each test case.

            Format:
            1. **Test Scenario**: <scenario details>
               **Test Steps**: <steps>
               **Pre Requisites**: <conditions>
               **Expected Results**: <expected results>
               **Pass/Fail Criteria**: <criteria>
            ### END TEST CASE ###
            [/INST]
            """.format(error=error_message, solution=solution, count=missing_neg)
            neg_template = ChatPromptTemplate.from_template(neg_prompt)
            formatted_neg = neg_template.format_prompt().to_messages()
            neg_response = llm.invoke(formatted_neg).content.strip()
            generated_neg = [tc.strip() for tc in re.split(r"\n### END TEST CASE ###\n", neg_response) if tc.strip()]
            generated_neg = [f"**Generated Negative Test Case:**\n{tc}" for tc in generated_neg][:missing_neg]

        # Combine CSV and generated test cases for each type
        final_pos = csv_test_cases_pos + generated_pos
        final_neg = csv_test_cases_neg + generated_neg

        # Ensure exactly 2 of each type (if more than 2 exist, take the first two)
        final_pos = final_pos[:2]
        final_neg = final_neg[:2]

        # Format final output
        test_cases_text = "\n\n".join(final_pos + final_neg)

        response_template = (
            "**Error:**\n{Error}\n\n"
            "**Solution:**\n{Solution}\n\n"
            "**Explanation:**\n{Explanation}\n\n"
            "**Final Test Cases (2 Positive and 2 Negative):**\n{TestCases}"
        )
        return {"response": response_template.format(
            Error=error_message,
            Solution=solution,
            Explanation=explanation,
            TestCases=test_cases_text
        )}
    except Exception as e:
        logging.error("Validation/Generation error: %s", str(e))
        return {"response": f"Error processing request: {str(e)}"}

# --------------------------------------------------------------------
# Build the State Graph Workflow
# --------------------------------------------------------------------
workflow = StateGraph(AgentState)
workflow.add_node("retrieve", lambda state: {"context": retriever.invoke(state["input"])})
workflow.add_node("validate_or_generate_test_cases", validate_or_generate_test_cases)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "validate_or_generate_test_cases")
workflow.add_edge("validate_or_generate_test_cases", END)
agent = workflow.compile()

# --------------------------------------------------------------------
# Automated Evaluation & Self-improvement Functions (unchanged)
# --------------------------------------------------------------------
def auto_evaluate_solution(response: str) -> int:
    # Check if our explicit delimiter is present in the generated parts
    if "### END TEST CASE ###" in response or "Generated" in response:
        return 5
    elif "**Error**:" in response:
        return 1
    else:
        return 3

def generate_alternative_solution(error_message: str) -> str:
    alt_prompt = """
    [INST] Provide a concise, actionable alternative solution for the following error:
    Error: {error}
    Ensure that the solution is clear and does not include any follow-up questions.
    [/INST]
    """
    alt_template = ChatPromptTemplate.from_template(alt_prompt)
    formatted_alt = alt_template.format_prompt(error=error_message)
    alternative_solution = llm.invoke(formatted_alt.to_messages()).content.strip()

    test_case_prompt = """
    [INST] Given the error and the alternative solution:
    Error: {error}
    Solution: {solution}
    Generate EXACTLY 4 structured test cases (2 positive and 2 negative) with the delimiter "### END TEST CASE ###" after each test case.
    Each test case must include:
      - **Test Scenario**
      - **Test Steps**
      - **Pre Requisites**
      - **Expected Results**
      - **Pass/Fail Criteria**
    [/INST]
    """
    tc_template = ChatPromptTemplate.from_template(test_case_prompt)
    formatted_tc = tc_template.format_prompt(error=error_message, solution=alternative_solution)
    alternative_test_cases = llm.invoke(formatted_tc.to_messages()).content.strip()

    alt_response = (
        "**Alternative Solution (Generated):**\n{AltSolution}\n\n"
        "**Test Cases for Alternative Solution:**\n{AltTestCases}"
    ).format(
        AltSolution=alternative_solution,
        AltTestCases=alternative_test_cases
    )
    return alt_response

def get_solution_autonomously(error_message: str) -> str:
    max_iterations = 3
    iteration = 0
    while iteration < max_iterations:
        logging.info("Iteration %d: Processing error: %s", iteration + 1, error_message)
        result = agent.invoke({"input": error_message.strip()})
        response = result["response"]
        logging.info("Agent response:\n%s", response)
        rating = auto_evaluate_solution(response)
        logging.info("Auto-evaluated rating: %d", rating)
        if rating < 3:
            logging.info("Rating below threshold. Generating alternative solution.")
            alt_response = generate_alternative_solution(error_message)
            logging.info("Alternative response generated.")
            return alt_response
        else:
            return response
        iteration += 1
    logging.info("Max iterations reached. Returning last response.")
    return response

# --------------------------------------------------------------------
# Autonomous Agent Execution
# --------------------------------------------------------------------
def main():
    error_description = "Search results not displaying correctly"
    final_solution = get_solution_autonomously(error_description)
    print("\n=== Final Autonomous Response ===\n")
    print(final_solution)

if __name__ == "__main__":
    main()



=== Final Autonomous Response ===

**Error:**
Search results not displaying correctly

**Solution:**
Updated Elasticsearch query logic to handle special characters.

**Explanation:**
The error "Search results not displaying correctly" often arises when Elasticsearch struggles to interpret search queries containing special characters. 

Here's why the solution "Updated Elasticsearch query logic to handle special characters" fixes the issue:

* **Special characters as wildcards:**  Many special characters (like `*`, `?`, `[`, `]`, `~`) have special meanings in regular expressions, which Elasticsearch uses for querying.  If your search query includes these characters without proper escaping, Elasticsearch might misinterpret them as part of the regular expression, leading to unexpected or incomplete search results.

* **Encoding issues:**  Different character encodings can cause problems. If your data is encoded differently than Elasticsearch expects, special characters might be displayed